In [ ]:
from tdc.benchmark_group import admet_group
import pickle as pk
import numpy as np

group = admet_group(path = 'data/')
names = []

for bench in group:
    names.append(bench['name'])

models = [ 
         'scratch_20L_wide_def_2e-5_16p', 
         'qm_all_20L_wide_def_2e-5_16p', 
         'charges_20L_wide_def_2e-5_16p', 
         'nmr_20L_wide_def_2e-5_16p', 
         'fukui_n_20L_wide_def_2e-5_16p', 
         'fukui_e_20L_wide_def_2e-5_16p',
         'masking_20L_wide_def_2e-5_16p',
         'homo-lumo_20L_wide_def_2e-5_16p',
        ]

group = admet_group(path = 'data/')

results_collection = {}
metric_dict = {}

for name in names:
    results_collection[name] = {}
    
    for model in models:
        results_collection[name][model] = []
        
        for seed in [1, 2, 3, 4, 5]:
    
            benchmark = group.get(name) 

            predictions = {}

            with open(f'./TDC_checkpoints/{model}/{name}_{seed}/predictions_2.pkl', 'rb') as f:
                t = pk.load(f)

            predictions[name] = np.array(t)
            results = group.evaluate(predictions)
            results_collection[name][model].append(results[name][list(results[name].keys())[0]])
            metric_dict[name] = list(results[name].keys())[0]

In [ ]:
results_collection

In [ ]:
mean_results = {}
for name in names:
    mean_results[name] = {}
    for model in models:
        temp = []
        for seed in [1,2,3,4,5]:
            temp.append(results_collection[name][model][seed-1])
        mean = np.mean(temp)
        mean_results[name][model] = mean

In [ ]:
from scipy.stats import ttest_rel
import numpy as np
from matplotlib import pyplot as plt
from tdc.benchmark_group import admet_group

group = admet_group(path = 'data/')
names = []

for bench in group:
    names.append(bench['name'])
    
ttest_matrices = {}
ordered_models = {}

for name in names:
    
    metric = metric_dict[name]
    
    t_test_matrix = []
    
    sorting_idx = np.argsort([mean_results[name][k] for k in models])
    
    if metric!='mae':
        sorting_idx = np.flip(sorting_idx)
    
    ordered_models[name] = np.array(models)[sorting_idx]
    
    for model in np.array(models)[sorting_idx]:
        
        tmp = []
        
        for model_ in np.array(models)[sorting_idx]:
            if model!=model_:
                tmp.append(ttest_rel(results_collection[name][model], results_collection[name][model_], alternative = 'greater').pvalue)
            else:
                tmp.append(np.mean(results_collection[name][model]))
        
        t_test_matrix.append(tmp)
        
    ttest_matrices[name] = t_test_matrix

In [ ]:
def check_significant(matrix):
    matrix = np.array(matrix)
    mask = matrix>=0.05
    matrix[mask]= 0
    matrix[~mask] = 1
    return matrix

name = names[10]
print(name)
plt.imshow(check_significant(ttest_matrices[name]))
plt.yticks(np.arange(0,len(ordered_models[name])), ordered_models[name])
plt.xticks(np.arange(0,len(ordered_models[name])), ordered_models[name], rotation = 90)